## Draw mask with this upper part of the notebook:

In [ ]:
%gui qt

import napari
from bg_atlasapi import BrainGlobeAtlas

atlas = BrainGlobeAtlas("mpin_zfish_1um")

v = napari.view_image(atlas.reference)

In [ ]:
lab_stack = v.layers["Labels"].data

In [ ]:
import numpy as np

In [ ]:
np.save("/Users/luigipetrucco/Desktop/mask.npy", lab_stack)

## Find neurons here

In [ ]:
%matplotlib widget
from pathlib import Path

import numpy as np
import pandas as pd
from bg_atlasapi import BrainGlobeAtlas
from matplotlib import pyplot as plt
from morphapi.api.mpin_celldb import MpinMorphologyAPI
from tqdm import tqdm

atlas = BrainGlobeAtlas("mpin_zfish_1um")

api = MpinMorphologyAPI()
mask = np.load("/Users/luigipetrucco/Desktop/mask.npy")

neurons = []
c = 0
for neuron in tqdm(api.neurons_df.index):
    try:
        neurons.append(api.load_neurons(neuron)[0])
    except:
        c += 1

print("problems on ", c, " files, loaded ", len(neurons))

In [ ]:
path = Path("/Users/luigipetrucco/.morphapi/Data/mpin_morphology/fixed")


neurons_dfs = []
somas = []
valid = []
for k, neuron in enumerate(path.glob("*.swc")):
    neuron = pd.read_csv(neuron, sep=" ", header=None, comment="#")
    soma = neuron.iloc[0, 2:5].values

    try:
        if mask[atlas._idx_from_coords(soma, False)] > 0:
            valid.append(k)
    except IndexError:
        pass

    somas.append(soma)
    neurons_dfs.append(neuron)

In [ ]:
all_somas = np.array(somas)

In [ ]:
plt.figure()
plt.scatter(all_somas[:, 0], all_somas[:, 1])
plt.scatter(all_somas[valid, 0], all_somas[valid, 1])

In [ ]:
sel_neurons = [neurons_dfs[k].values[:, 2:-2] for k in valid]
n_neurons = len(sel_neurons)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

cell_dist = np.zeros((n_neurons,) * 2)

for i in tqdm(range(n_neurons)):
    for j in range(i + 1, n_neurons):
        c1 = sel_neurons[i]  # coords of cell 1
        c2 = sel_neurons[j]  # coords of cell 2
        coords_dist = euclidean_distances(
            c1, c2
        )  # calculate pairwise Euclidean distance
        d = np.median(np.concatenate([np.min(coords_dist, 0), np.min(coords_dist, 1)]))
        # d = np.median(np.concatenate([np.median(coords_dist, 0), np.median(coords_dist, 1)]))
        cell_dist[i, j], cell_dist[j, i] = d, d
        # cell_dist[i, j] = np.median(np.min(coords_dist, 0))  # median of min distances for each coord cell 1
        # cell_dist[j, i] = np.median(np.min(coords_dist, 1)) # median of min distances for each coord cell

In [ ]:
for i in range(n_neurons):
    for j in range(n_neurons):
        cell_dist[j, i] = cell_dist[i, j]

In [ ]:
import scipy.cluster.hierarchy as sch
import seaborn as sns

linkage = sch.linkage(cell_dist, method="average")

In [ ]:
f = plt.figure(figsize=(5, 9))
f.add_axes((0.2, 0.6, 0.6, 0.3))
dendrogram = sch.dendrogram(linkage)
s_idxs = dendrogram["leaves"]
# plt.xticks([])
# plt.yticks([])

f.add_axes((0.2, 0.2, 0.6, 0.5))
plt.imshow(cell_dist[s_idxs, :][:, s_idxs])
# plt.xticks([])
# plt.yticks([])
plt.show()
# sns.despine(left=True, bottom=True)

In [ ]:
plt.figure(figsize=(6, 3))
thr = 200
dendrogram = sch.dendrogram(linkage)
plt.axhline(thr)

In [ ]:
clusters = sch.cut_tree(linkage, height=thr).flatten()

In [ ]:
cols = sns.color_palette(n_colors=clusters.max() + 1)

In [ ]:
plt.figure(figsize=(6, 3))
# for i in range(clusters.max() + 1):  # [0]
# n_val = n[(n[:, 1] > 100) & (n[:, 2] > 100), :]
for c, n in zip(clusters, sel_neurons):
    n = n[n[:, 0] < 800, :]
    plt.scatter(n[:, 0], n[:, 2], s=2, color=cols[c])
    plt.axis("equal")

plt.xlabel("A-P")
plt.ylabel("L-R")
plt.tight_layout()

In [ ]:
-150 + 312  # +150

In [ ]:
n_clusts

In [ ]:
n_clusts = clusters.max() + 1
f, axs = plt.subplots(n_clusts // 4, 4, figsize=(3, 8), sharex=True)


bins = np.arange(-150, 150, 10)
for c in range(n_clusts):
    ax = axs[c // 4, c % 4]

    thiss = np.concatenate(
        [sel_neurons[i] for i in range(len(clusters)) if clusters[i] == c]
    )
    thiss = thiss[thiss[:, 0] < 800, :]
    ax.hist(thiss[:, 2] - 312, bins, lw=0, fc=cols[c], alpha=0.3, density=True)
    ax.axvline(0)
    ax.set(yticks=[], ylabel=c)

sns.despine

In [ ]:
2, 4, 11, 19, 20, 17

In [ ]:
%gui qt

import napari

In [ ]:
napari.view_points

In [ ]:
np.array(cols[c])

In [ ]:
thiss.shape

In [ ]:
v = napari.Viewer(ndisplay=3)
loop_over = [2, 4, 11, 19, 20, 17]  # range(clusters.max() + 1)

for c in loop_over:
    thiss = np.concatenate(
        [sel_neurons[i][:1, :] for i in range(len(clusters)) if clusters[i] == c]
    )
    this_col = np.array(cols[c])
    v.add_points(
        thiss,
        name=str(c) + "_soma",
        edge_width=0,
        size=15,
        face_color=this_col,
        n_dimensional=True,
    )

for c in loop_over:
    thisc = np.concatenate(
        [sel_neurons[i] for i in range(len(clusters)) if clusters[i] == c]
    )
    this_col = np.array(cols[c])
    v.add_points(
        thisc,
        name=str(c),
        edge_width=0,
        size=3,
        face_color=this_col,
        n_dimensional=True,
    )

In [ ]:
v.add_image(atlas.reference)

In [ ]:
v.add_image(mask)